## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов.
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import pymorphy2
import numpy as np
import re
import pandas as pd
import gensim

In [2]:
NEWS_PATH = '../data/news/'
TRAIN_PATH = NEWS_PATH + 'news_train.txt'
TEST_PATH = NEWS_PATH + 'news_test.txt'

In [3]:
train_lines = list(open(TRAIN_PATH, 'r', encoding='utf-8'))
test_lines = list(open(TEST_PATH, 'r', encoding='utf-8'))

In [4]:
import random
random.shuffle(train_lines)

In [5]:
len(train_lines), train_lines[0].split('\t')
# тема
# заголовок
# содержание

(15000,
 ['science',
  'Boeing займется техобслуживанием тайваньских "Апачей"',
  'Американский авиастроительный концерн Boeing получил контракт на техническое обслуживание ударных вертолетов AH-64D Apache, которые в перспективе должны получить сухопутные войска Тайваня. Как сообщает Defense Aerospace, сумма сделки составила 141,3 миллиона долларов. Речь о идет о техническом обслуживании 30 машин. Подробности соглашения не раскрываются. Как ожидается, работы по контракту завершатся в декабре 2017 года.Тайвань заказал у США 30 вертолетов Apache в 2008 году. Помимо этих машин в заказ также вошла поставка многоцелевых вертолетов Black Hawk, зенитных ракетных комплексов Patriot и истребителей F-16. В начале 2010 года стало известно, что США частично одобрили продажу военной техники Тайваню. В частности, было решено поставить стране комплексы Patriot, вертолеты Black Hawk и оборудование для связи.Считалось, что поставка AH-64D Тайваню была заблокирована, однако в 2010 году стало известно, ч

In [6]:
from collections import Counter

Counter([line.split('\t')[0] for line in train_lines])

Counter({'science': 2156,
         'sport': 2215,
         'economics': 2080,
         'media': 2111,
         'culture': 2053,
         'life': 2033,
         'travel': 289,
         'style': 284,
         'forces': 1225,
         'business': 554})

In [ ]:
'https://drive.google.com/file/d/1mG3tPS_59pANrgwd6T2IgnHWgph4vYbg/view?usp=sharing'

### 1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)

In [7]:
def tokenize_string(string, regex_string=r'\b\w+\b'):
    return re.findall(regex_string, string.lower())

In [12]:
def tokenize_dataset(lines, regex_string=r'\b\w+\b'):
    result = []
    for line in lines:
        label, title, text = line.split('\t')
        
        title = tokenize_string(title, regex_string)
        text = tokenize_string(text, regex_string)
        
        result.append(dict())
        result[-1]['label'] = label
        result[-1]['title'] = title
        result[-1]['text'] = text
        
    return result

In [9]:
from tqdm.notebook import tqdm

In [14]:
def lemmatize_dataset(lines):
    morph_analyzer = pymorphy2.MorphAnalyzer()
    for line in tqdm(lines):
        line['title'] = list(map(lambda x: morph_analyzer.parse(x)[0].normal_form, line['title']))
        line['text'] = list(map(lambda x: morph_analyzer.parse(x)[0].normal_form, line['text']))

In [13]:
train_data = tokenize_dataset(train_lines)
train_data[0]

{'label': 'science',
 'title': ['boeing', 'займется', 'техобслуживанием', 'тайваньских', 'апачей'],
 'text': ['американский',
  'авиастроительный',
  'концерн',
  'boeing',
  'получил',
  'контракт',
  'на',
  'техническое',
  'обслуживание',
  'ударных',
  'вертолетов',
  'ah',
  '64d',
  'apache',
  'которые',
  'в',
  'перспективе',
  'должны',
  'получить',
  'сухопутные',
  'войска',
  'тайваня',
  'как',
  'сообщает',
  'defense',
  'aerospace',
  'сумма',
  'сделки',
  'составила',
  '141',
  '3',
  'миллиона',
  'долларов',
  'речь',
  'о',
  'идет',
  'о',
  'техническом',
  'обслуживании',
  '30',
  'машин',
  'подробности',
  'соглашения',
  'не',
  'раскрываются',
  'как',
  'ожидается',
  'работы',
  'по',
  'контракту',
  'завершатся',
  'в',
  'декабре',
  '2017',
  'года',
  'тайвань',
  'заказал',
  'у',
  'сша',
  '30',
  'вертолетов',
  'apache',
  'в',
  '2008',
  'году',
  'помимо',
  'этих',
  'машин',
  'в',
  'заказ',
  'также',
  'вошла',
  'поставка',
  'много

In [15]:
lemmatize_dataset(train_data)
train_data[0]

{'label': 'science',
 'title': ['boeing', 'заняться', 'техобслуживание', 'тайваньский', 'апач'],
 'text': ['американский',
  'авиастроительный',
  'концерн',
  'boeing',
  'получить',
  'контракт',
  'на',
  'технический',
  'обслуживание',
  'ударный',
  'вертолёт',
  'ah',
  '64d',
  'apache',
  'который',
  'в',
  'перспектива',
  'должный',
  'получить',
  'сухопутный',
  'войско',
  'тайвань',
  'как',
  'сообщать',
  'defense',
  'aerospace',
  'сумма',
  'сделка',
  'составить',
  '141',
  '3',
  'миллион',
  'доллар',
  'речь',
  'о',
  'идти',
  'о',
  'технический',
  'обслуживание',
  '30',
  'машина',
  'подробность',
  'соглашение',
  'не',
  'раскрываться',
  'как',
  'ожидаться',
  'работа',
  'по',
  'контракт',
  'завершиться',
  'в',
  'декабрь',
  '2017',
  'год',
  'тайвань',
  'заказать',
  'у',
  'сша',
  '30',
  'вертолёт',
  'apache',
  'в',
  '2008',
  'год',
  'помимо',
  'этот',
  'машина',
  'в',
  'заказ',
  'также',
  'войти',
  'поставка',
  'многоцелевой

### 2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. 
Можно использовать [gensim] (https://radimrehurek.com/gensim/models/word2vec.html). Продемонстрировать семантические ассоциации. 

In [16]:
train_df = pd.DataFrame(train_data)
train_df

,label,title,text
0,science,"[boeing, заняться, техобслуживание, тайваньски...","[американский, авиастроительный, концерн, boei..."
1,sport,"[сергей, карякин, выиграть, кубок, мир, по, ша...","[российский, шахматист, сергей, карякин, вперв..."
2,sport,"[врач, разрешить, пеле, зажечь, олимпийский, о...","[врач, разрешить, бывший, футболист, сборная, ..."
3,economics,"[евросоюз, выделить, 125, миллион, евро, на, п...","[из, бюджет, евросоюз, выделить, 125, миллион,..."
4,media,"[сми, сообщить, о, продажа, павел, дуров, акци...","[основатель, и, генеральный, директор, социаль..."
...,...,...,...
14995,forces,"[бывший, президент, анталбанка, арестовать, по...","[бывший, президент, анталбанка, магомед, мухий..."
14996,life,"[британец, отказаться, покупать, книга, премье...","[книга, премьер, министр, великобритания, горд..."
14997,economics,"[дочка, башнефть, лишить, лицензия, на, местор...","[роснедра, 18, май, отменить, свой, приказ, о,..."
14998,media,"[угадать, мелодия, вернуться, в, эфир]","[программа, угадать, мелодия, в, январь, 2013,..."


In [17]:
word2vec = gensim.models.Word2Vec(train_df['text'].append(train_df['title']))

In [18]:
print(word2vec.wv.most_similar(positive=['овечкин']), '\n')
print(word2vec.wv.most_similar(positive=['samsung']), '\n')
print(word2vec.wv.most_similar(positive=['спартак']), '\n')
print(word2vec.wv.most_similar(positive=['ios']), '\n')
print(word2vec.wv.most_similar(positive=['владивосток']), '\n')

[('малкин', 0.9228668212890625), ('питтсбург', 0.9222313165664673), ('голкипер', 0.9207319021224976), ('кириленко', 0.9102184772491455), ('испанец', 0.910172700881958), ('радулов', 0.9056620001792908), ('аргентинец', 0.8954154849052429), ('вратарь', 0.8910346627235413), ('надаль', 0.8870075941085815), ('пингвинс', 0.8808801174163818)] 

[('apple', 0.8774051666259766), ('lg', 0.8682652115821838), ('motorola', 0.8467772603034973), ('смартфон', 0.8339815735816956), ('планшет', 0.8321726322174072), ('nokia', 0.8281078934669495), ('microsoft', 0.8219476342201233), ('iphone', 0.8164066076278687), ('мобильный', 0.8061249256134033), ('htc', 0.8052313327789307)] 

[('динамо', 0.9529554843902588), ('цска', 0.9505517482757568), ('локомотив', 0.9247673749923706), ('зенит', 0.8965145945549011), ('анжи', 0.8618971705436707), ('полузащитник', 0.8587614297866821), ('челси', 0.8583778142929077), ('ска', 0.8508938550949097), ('краснодар', 0.8498367071151733), ('армеец', 0.8480013608932495)] 

[('android

![image.png](https://sun9-14.userapi.com/impg/fz61Au_EA7v-8IQDq_ZEDqAc_ao4NNj44rd3CQ/vu1EETE-dG8.jpg?size=640x332&quality=96&proxy=1&sign=9835fa79d633abb6cfa2ef0495b8e0a5&type=album)

### 3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. 
Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия

In [19]:
mapping_dict = {'sport': 0,
                'culture': 1,
                'science': 2,
                'media': 3,
                'economics': 4,
                'life': 5,
                'forces': 6,
                'travel': 7,
                'style': 8,
                'business': 9}

In [20]:
def map_labels(labels, mapping_dict):
    return labels.map(lambda label: mapping_dict[label]).to_list()

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [22]:
def mean_tfidf_vectorizer(X, word2vec):
    word2vec_dict = dict(zip(word2vec.wv.index2word, word2vec.wv.vectors))
    dim = len(next(iter(word2vec_dict.values())))        
    
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(X)
    max_idf = max(tfidf.idf_)
    word2weight = defaultdict(
        lambda: max_idf, 
        [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
    return np.array([
        np.mean([word2vec[w] * word2weight[w] 
                 for w in words if w in word2vec] or 
                [np.zeros(dim)], axis=0) 
        for words in X
    ])

In [23]:
# Train data
x_train = mean_tfidf_vectorizer(train_df['text'], word2vec)
y_train = map_labels(train_df['label'], mapping_dict)

C:\Users\tsark\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\tsark\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [24]:
# Test data
test_lines = list(open(TEST_PATH, 'r', encoding='utf-8'))
test_data = tokenize_dataset(test_lines)

lemmatize_dataset(test_data)
test_df = pd.DataFrame(test_data)

x_test = mean_tfidf_vectorizer(test_df['text'], word2vec)
y_test = map_labels(test_df['label'], mapping_dict)

C:\Users\tsark\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\tsark\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [26]:
clf = LogisticRegression(n_jobs = -1)
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
accuracy_score(y_test, predictions)

0.8413333333333334

In [27]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [28]:
clf = SVC()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
accuracy_score(y_test, predictions)

0.8516666666666667